# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kalulushi,-12.8378,28.0919,15.14,52,18,2.18,ZM,07/03/2024
1,1,dudinka,69.4058,86.1778,13.25,54,100,7.12,RU,07/03/2024
2,2,nadym,65.5333,72.5167,12.99,90,79,4.60,RU,07/03/2024
3,3,adamstown,-25.0660,-130.1015,20.72,64,100,5.13,PN,07/03/2024
4,4,kangding,30.0513,101.9603,17.04,98,100,0.19,CN,07/03/2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500  
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
select_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 30) & 
(city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] <= 20)]

# Drop any rows with null values
select_cities_df = select_cities_df.dropna()

# Display sample data
select_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,arraial do cabo,-22.9661,-42.0278,22.92,78,0,1.54,BR,07/03/2024
54,54,melilla,35.2937,-2.9383,25.21,69,0,1.54,ES,07/03/2024
74,74,port-vila,-17.7338,168.3219,21.78,100,20,0.51,VU,07/03/2024
86,86,prado,-17.3411,-39.2208,23.09,68,0,3.10,BR,07/03/2024
90,90,trairi,-3.2778,-39.2689,24.37,90,17,3.47,BR,07/03/2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = select_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df.head()

/var/folders/tn/g6qz3f296z7cn5d90mgpvj600000gn/T/ipykernel_29090/1610697898.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = None


,City,Country,Lat,Lng,Humidity,Hotel Name
50,arraial do cabo,BR,-22.9661,-42.0278,78,None
54,melilla,ES,35.2937,-2.9383,69,None
74,port-vila,VU,-17.7338,168.3219,100,None
86,prado,BR,-17.3411,-39.2208,68,None
90,trairi,BR,-3.2778,-39.2689,90,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arraial do cabo - nearest hotel: No hotel found
melilla - nearest hotel: Rusadir
port-vila - nearest hotel: Holiday Inn Resort Vanuatu
prado - nearest hotel: Pousada Recanto do Prado
trairi - nearest hotel: Pousada Villa Aurora
praia da vitoria - nearest hotel: Salles
bani walid - nearest hotel: فندق الزيتونة
antonio enes - nearest hotel: Hotel Quirimbas
camapua - nearest hotel: No hotel found
muynoq - nearest hotel: Lala
muroto-misakicho - nearest hotel: 民宿うらしま
tripoli - nearest hotel: الاصيل
uruzgan - nearest hotel: No hotel found
tuzlukcu - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
princeville - nearest hotel: No hotel found
mahina - nearest hotel: Motu Martin
joao pinheiro - nearest hotel: Lícia
cantilan - nearest hotel: Karajaw Hotel
rancharia - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
machico - nearest hotel: Residence Inn
uychi - nearest hotel: No hotel found
ribeira 

,City,Country,Lat,Lng,Humidity,Hotel Name
50,arraial do cabo,BR,-22.9661,-42.0278,78,No hotel found
54,melilla,ES,35.2937,-2.9383,69,Rusadir
74,port-vila,VU,-17.7338,168.3219,100,Holiday Inn Resort Vanuatu
86,prado,BR,-17.3411,-39.2208,68,Pousada Recanto do Prado
90,trairi,BR,-3.2778,-39.2689,90,Pousada Villa Aurora
96,praia da vitoria,PT,38.7333,-27.0667,75,Salles
105,bani walid,LY,31.7566,13.9942,56,فندق الزيتونة
106,antonio enes,MZ,-16.2325,39.9086,73,Hotel Quirimbas
115,camapua,BR,-19.5314,-54.0439,25,No hotel found
150,muynoq,UZ,43.7683,59.0214,27,Lala


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)